In [59]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_selection import chi2
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn import neighbors
from sklearn import svm

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [60]:
import warnings
warnings.filterwarnings("ignore")

# Data

In [61]:
train = pd.read_csv("voting_train.csv")
test = pd.read_csv("voting_test.csv")

In [62]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 218 entries, 0 to 217
Data columns (total 18 columns):
 #   Column                                  Non-Null Count  Dtype 
---  ------                                  --------------  ----- 
 0   ID                                      218 non-null    int64 
 1   class                                   218 non-null    object
 2   handicapped-infants                     218 non-null    object
 3   water-project-cost-sharing              218 non-null    object
 4   adoption-of-the-budget-resolution       218 non-null    object
 5   physician-fee-freeze                    218 non-null    object
 6   el-salvador-aid                         218 non-null    object
 7   religious-groups-in-schools             218 non-null    object
 8   anti-satellite-test-ban                 218 non-null    object
 9   aid-to-nicaraguan-contras               218 non-null    object
 10  mx-missile                              218 non-null    object
 11  immigr

In [63]:
train.head()

,ID,class,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missile,immigration,synfuels-crporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
0,67,republican,n,y,n,y,y,y,y,n,n,n,y,y,y,y,n,y
1,338,democrat,y,n,y,n,n,n,y,y,y,n,n,n,n,n,y,y
2,35,democrat,y,y,y,n,n,n,y,y,y,n,n,n,n,n,y,y
3,122,republican,n,unknown,n,y,y,y,n,n,n,y,n,y,y,y,n,y
4,420,democrat,y,y,y,n,n,n,y,y,y,n,n,n,n,n,n,y


In [64]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 217 entries, 0 to 216
Data columns (total 17 columns):
 #   Column                                  Non-Null Count  Dtype 
---  ------                                  --------------  ----- 
 0   ID                                      217 non-null    int64 
 1   handicapped-infants                     217 non-null    object
 2   water-project-cost-sharing              217 non-null    object
 3   adoption-of-the-budget-resolution       217 non-null    object
 4   physician-fee-freeze                    217 non-null    object
 5   el-salvador-aid                         217 non-null    object
 6   religious-groups-in-schools             217 non-null    object
 7   anti-satellite-test-ban                 217 non-null    object
 8   aid-to-nicaraguan-contras               217 non-null    object
 9   mx-missile                              217 non-null    object
 10  immigration                             217 non-null    object
 11  synfue

In [65]:
test.head()

,ID,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missile,immigration,synfuels-crporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
0,430,y,n,y,n,unknown,n,y,y,y,y,n,y,n,unknown,y,y
1,411,n,n,n,y,y,y,n,n,n,n,y,y,y,y,n,y
2,167,y,n,y,y,y,y,y,y,n,y,n,y,n,y,y,y
3,99,y,y,y,n,n,y,y,y,y,y,n,n,n,n,n,y
4,415,y,y,y,n,n,n,y,y,y,n,n,n,n,n,n,y


In [66]:
X = train.drop(["ID", "class"], axis=1)
y = train["class"]

X_test = test.drop(["ID"], axis=1)

# Pre-processing

## Encoding

In [ ]:
ohe = OneHotEncoder()
X_enc = ohe.fit_transform(X)
X_test_enc = ohe.fit_transform(X_test)

# Exploratory data analysis

In [67]:
X.isnull().sum()

handicapped-infants                       0
water-project-cost-sharing                0
adoption-of-the-budget-resolution         0
physician-fee-freeze                      0
el-salvador-aid                           0
religious-groups-in-schools               0
anti-satellite-test-ban                   0
aid-to-nicaraguan-contras                 0
mx-missile                                0
immigration                               0
synfuels-crporation-cutback               0
education-spending                        0
superfund-right-to-sue                    0
crime                                     0
duty-free-exports                         0
export-administration-act-south-africa    0
dtype: int64

In [ ]:
chi2(X_enc, y)

(array([2.25721586e+01, 2.13390681e-01, 2.92925171e+01, 2.00556506e-01,
        1.24492043e+00, 8.87962151e-01, 6.76980225e+01, 8.80860215e-02,
        4.98858134e+01, 8.21075486e+01, 1.04961720e+00, 1.00947269e+02,
        6.61028607e+01, 2.13390681e-01, 5.84342577e+01, 2.98231336e+01,
        2.30401229e+00, 1.71709683e+01, 2.43667425e+01, 6.11794504e-01,
        1.99391226e+01, 4.99767559e+01, 5.30444240e+00, 4.76090131e+01,
        3.92534643e+01, 4.60802458e+00, 3.61429076e+01, 8.87962151e-01,
        7.06781362e-01, 5.51944647e-01, 1.20025524e+01, 2.64258065e-01,
        2.43658973e+01, 5.14933775e+01, 3.52344086e-01, 6.39304719e+01,
        3.26084899e+01, 6.48535980e-02, 2.72403550e+01, 5.83132540e+01,
        1.28726882e+00, 3.46754890e+01, 2.12920913e+01, 1.84366452e+00,
        3.34059596e+01, 2.06855367e+01, 9.22156272e+00, 1.10922433e-01]),
 array([2.02396523e-06, 6.44122705e-01, 6.22355606e-08, 6.54272025e-01,
        2.64524870e-01, 3.46030144e-01, 1.90554563e-16, 7.6662

# Model selection

In [44]:
# Simple Grid Search to find best hyperparameters for differnet classifier

cv = 5          # number of folds
verbose = 1     # information shown during training

## KNN

In [45]:
parameters = {
"n_neighbors":[1, 5, 10, 20], 
"weights":["uniform", "distance"],
"metric":["euclidean", "manhattan", "chebyshev", "minkowski", "wminkowski", "seuclidean", "mahalanobis"]}
knn = GridSearchCV(neighbors.KNeighborsClassifier(), parameters, cv=cv, verbose=verbose)
knn.fit(X_enc, y)

knn_results = pd.DataFrame(knn.cv_results_)
knn_results= knn_results[["param_n_neighbors", "param_weights", "param_metric", "mean_test_score"]]    
knn_results.sort_values(["mean_test_score"], ascending=False).head(10)

Fitting 5 folds for each of 56 candidates, totalling 280 fits


,param_n_neighbors,param_weights,param_metric,mean_test_score
0,1,uniform,euclidean,0.930973
24,1,uniform,minkowski,0.930973
2,5,uniform,euclidean,0.930973
3,5,distance,euclidean,0.930973
27,5,distance,minkowski,0.930973
8,1,uniform,manhattan,0.930973
9,1,distance,manhattan,0.930973
10,5,uniform,manhattan,0.930973
11,5,distance,manhattan,0.930973
1,1,distance,euclidean,0.930973


## Decision Tree

In [46]:
parameters = {
    "criterion":["gini", "entropy"],
    "splitter":["best", "random"], 
    "max_depth":["None", 5, 10, 20],
    "max_features":["auto", "sqrt", "log2"]}    
dt = GridSearchCV(DecisionTreeClassifier(), parameters, cv=cv, verbose=verbose)
dt.fit(X_enc, y)

dt_results = pd.DataFrame(dt.cv_results_)
dt_results = dt_results[["param_criterion", "param_splitter", "param_max_depth", "param_max_features", "mean_test_score"]]
dt_results.sort_values(["mean_test_score"], ascending=False).head(10)

Fitting 5 folds for each of 48 candidates, totalling 240 fits


,param_criterion,param_splitter,param_max_depth,param_max_features,mean_test_score
37,entropy,random,10,auto,0.945032
30,entropy,best,5,auto,0.940381
20,gini,best,20,sqrt,0.935941
6,gini,best,5,auto,0.935624
7,gini,random,5,auto,0.931395
41,entropy,random,10,log2,0.931290
14,gini,best,10,sqrt,0.931290
32,entropy,best,5,sqrt,0.926956
34,entropy,best,5,log2,0.926744
31,entropy,random,5,auto,0.926744


## Random Forest

In [47]:
parameters = {
    "n_estimators":[10, 50, 100, 200, 300], 
    "criterion":["gini", "entropy"], 
    "max_depth":["None", 5, 10, 20],
    "max_features":["auto", "sqrt", "log2"]}
rf = GridSearchCV(RandomForestClassifier(), parameters, cv=cv, verbose=verbose)
rf.fit(X_enc, y)

rf_results = pd.DataFrame(rf.cv_results_)
rf_results = rf_results[["param_n_estimators", "param_criterion", "param_max_depth", "param_max_features", "mean_test_score"]]
rf_results.sort_values(["mean_test_score"], ascending=False).head(10)

Fitting 5 folds for each of 120 candidates, totalling 600 fits


,param_n_estimators,param_criterion,param_max_depth,param_max_features,mean_test_score
80,10,entropy,5,sqrt,0.967865
86,50,entropy,5,log2,0.967865
85,10,entropy,5,log2,0.967865
77,100,entropy,5,auto,0.963319
37,100,gini,10,sqrt,0.963319
40,10,gini,10,log2,0.963214
32,100,gini,10,auto,0.963214
41,50,gini,10,log2,0.958774
83,200,entropy,5,sqrt,0.958668
107,100,entropy,20,auto,0.958668


## SVM

In [48]:
parameters = {
    "C":[1, 5, 10],
    "kernel":["linear", "sigmoid", "rbf", "poly"],     
    "degree":[3, 5],
    "gamma":["scale", "auto"],
    "class_weight":["None", "balanced"]}
svm = GridSearchCV(svm.SVC(), parameters, cv=cv, verbose=verbose)
svm.fit(X_enc, y)

svm_results = pd.DataFrame(svm.cv_results_)
svm_results = svm_results[["param_C", "param_kernel", "param_degree", "param_gamma", "param_class_weight", "mean_test_score"]]
svm_results.sort_values(["mean_test_score"], ascending=False).head(10)

Fitting 5 folds for each of 96 candidates, totalling 480 fits


,param_C,param_kernel,param_degree,param_gamma,param_class_weight,mean_test_score
16,1,linear,3,scale,balanced,0.967970
20,1,linear,3,auto,balanced,0.967970
24,1,linear,5,scale,balanced,0.967970
28,1,linear,5,auto,balanced,0.967970
85,10,sigmoid,3,auto,balanced,0.963425
93,10,sigmoid,5,auto,balanced,0.963425
54,5,rbf,3,auto,balanced,0.963319
18,1,rbf,3,scale,balanced,0.963319
26,1,rbf,5,scale,balanced,0.963319
62,5,rbf,5,auto,balanced,0.963319


## Logistic Regression

In [49]:
parameters = {
    "penalty":["l1", "l2", "elasticnet", "none"], 
    "C":[1, 5, 10, 20],
    "class_weight":["None", "balanced"],
    "solver":["liblinear"]}
logreg = GridSearchCV(LogisticRegression(), parameters, cv=cv, verbose=verbose)
logreg.fit(X_enc, y)

logreg_results = pd.DataFrame(logreg.cv_results_)
logreg_results = logreg_results[["param_penalty", "param_C", "param_class_weight", "mean_test_score"]]
logreg_results.sort_values(["mean_test_score"], ascending=False).head(10)

Fitting 5 folds for each of 32 candidates, totalling 160 fits


,param_penalty,param_C,param_class_weight,mean_test_score
21,l2,10,balanced,0.963319
29,l2,20,balanced,0.963319
5,l2,1,balanced,0.963214
12,l1,5,balanced,0.958774
13,l2,5,balanced,0.958774
20,l1,10,balanced,0.954123
4,l1,1,balanced,0.954017
28,l1,20,balanced,0.949471
0,l1,1,None,NaN
1,l2,1,None,NaN


## Stochastic Gradient Descent

In [50]:
parameters = {
    "loss":["hinge", "log", "modified_huber", "squared_hinge", "perceptron"], 
    "penalty":["l2", "l1", "elasticnet"],
    "alpha":[0.0001, 0.001, 0.01]}
sgd = GridSearchCV(SGDClassifier(), parameters, cv=cv, verbose=verbose)
sgd.fit(X_enc, y)

sgd_results = pd.DataFrame(sgd.cv_results_)
sgd_results = sgd_results[["param_loss", "param_penalty", "param_alpha", "mean_test_score"]]
sgd_results.sort_values(["mean_test_score"], ascending=False).head(10)

Fitting 5 folds for each of 45 candidates, totalling 225 fits


,param_loss,param_penalty,param_alpha,mean_test_score
5,log,elasticnet,0.0001,0.972622
32,hinge,elasticnet,0.01,0.972622
44,perceptron,elasticnet,0.01,0.968076
30,hinge,l2,0.01,0.967970
35,log,elasticnet,0.01,0.967865
26,squared_hinge,elasticnet,0.001,0.967865
13,perceptron,l1,0.0001,0.967759
19,log,l1,0.001,0.963319
10,squared_hinge,l1,0.0001,0.958774
41,squared_hinge,elasticnet,0.01,0.958774


In [52]:
# convert results in markdown table

# result = knn_results
# print(result.sort_values(["mean_test_score"], ascending=False).head(20).to_markdown(index=False))

# Final model

In [54]:
model = SGDClassifier(loss="log", penalty="elasticnet", alpha=0.0001)
model.fit(X_enc, y)

SGDClassifier(loss='log', penalty='elasticnet')

In [55]:
predictions = pd.DataFrame(test["ID"])
predictions["class"] = model.predict(X_test_enc)

In [56]:
predictions.to_csv("submission.csv", index=False)